In [12]:
import re

In [13]:
# Let's read the contents of the file "sample_data/stderr.txt" in Python.
file_path = 'sample_data/stderr.txt'

# Reading the file and storing its contents in a variable.
with open(file_path, 'r') as file:
    sample_text = file.read()

sample_text

"\n> spc-llamathon@1.0.0 test\n> jest example.test.js\n\nFAIL ./example.test.js\n  add function\n    ✓ should add two positive numbers correctly (1 ms)\n    ✓ should add two negative numbers correctly\n    ✓ should add a positive and a negative number correctly\n    ✓ should add two zeros correctly\n    ✓ should add a number and zero correctly\n    ✓ should add two decimal numbers correctly\n    ✓ should add a decimal and an integer number correctly\n    ✓ should add two large numbers correctly\n    ✓ should return NaN when adding NaN\n    ✓ should return Infinity when adding Infinity\n    ✓ should return -Infinity when adding -Infinity (1 ms)\n  multiply\n    ✕ should multiply two positive numbers correctly\n    ✕ should multiply two negative numbers correctly\n    ✕ should multiply a positive and a negative number correctly\n    ✕ should multiply two zero numbers correctly\n    ✕ should multiply a number with zero correctly\n    ✕ should multiply a number with one correctly\n    ✕ sh

In [33]:
# Regular expression to recognize the start of a block and the end of the block
block_pattern = re.compile(r'(  .+?)(?:\n\s*\n)', re.DOTALL)

# Finding the first block in the sample text
match = block_pattern.search(sample_text)

# Get and print the first block
block = match.group(1)
print(block)

  add function
    ✓ should add two positive numbers correctly (1 ms)
    ✓ should add two negative numbers correctly
    ✓ should add a positive and a negative number correctly
    ✓ should add two zeros correctly
    ✓ should add a number and zero correctly
    ✓ should add two decimal numbers correctly
    ✓ should add a decimal and an integer number correctly
    ✓ should add two large numbers correctly
    ✓ should return NaN when adding NaN
    ✓ should return Infinity when adding Infinity
    ✓ should return -Infinity when adding -Infinity (1 ms)
  multiply
    ✕ should multiply two positive numbers correctly
    ✕ should multiply two negative numbers correctly
    ✕ should multiply a positive and a negative number correctly
    ✕ should multiply two zero numbers correctly
    ✕ should multiply a number with zero correctly
    ✕ should multiply a number with one correctly
    ✕ should multiply large numbers correctly
    ✕ should multiply decimal numbers correctly
    ✕ should r

In [37]:
res = {}
latest_suite = None

def handle_block(block):
    global latest_suite
    lines = block.split('\n')

    for line in lines:
        # detect if new suit
        if not line.startswith('   '):
            suit_name = line.strip()
            res[suit_name] = {}
            latest_suite = suit_name
            continue
        
        status = line[4]
        description = line[6:]

        # Regular expression to match and remove the " (...)" part at the end
        pattern = re.compile(r' \(.+\)$')

        # Remove the " (...)" part if present
        description = pattern.sub('', description)

        res[latest_suite][description] = False if status == '✕' else True

handle_block(block)
res

{'add function': {'should add two positive numbers correctly': True,
  'should add two negative numbers correctly': True,
  'should add a positive and a negative number correctly': True,
  'should add two zeros correctly': True,
  'should add a number and zero correctly': True,
  'should add two decimal numbers correctly': True,
  'should add a decimal and an integer number correctly': True,
  'should add two large numbers correctly': True,
  'should return NaN when adding NaN': True,
  'should return Infinity when adding Infinity': True,
  'should return -Infinity when adding -Infinity': True},
 'multiply': {'should multiply two positive numbers correctly': False,
  'should multiply two negative numbers correctly': False,
  'should multiply a positive and a negative number correctly': False,
  'should multiply two zero numbers correctly': False,
  'should multiply a number with zero correctly': False,
  'should multiply a number with one correctly': False,
  'should multiply large num

In [52]:
jest_path = 'sample_data/example.test.js'

# Reading the file and storing its contents in a variable.
with open(jest_path, 'r') as file:
    jest_text = file.read()

print(jest_text)

jest_lines = jest_text.split('\n')

// Import the target function
const add = require('./example').add;

describe('add function', () => {
    // Test case: Adding two positive numbers
    it('should add two positive numbers correctly', () => {
        expect(add(2, 3)).toBe(5);
    });

    // Test case: Adding two negative numbers
    it('should add two negative numbers correctly', () => {
        expect(add(-2, -3)).toBe(-5);
    });

    // Test case: Adding a positive and a negative number
    it('should add a positive and a negative number correctly', () => {
        expect(add(2, -3)).toBe(-1);
    });

    // Test case: Adding two zeros
    it('should add two zeros correctly', () => {
        expect(add(0, 0)).toBe(0);
    });

    // Test case: Adding a number and zero
    it('should add a number and zero correctly', () => {
        expect(add(5, 0)).toBe(5);
    });

    // Test case: Adding non-integer numbers
    it('should add non-integer numbers correctly', () => {
        expect(add(2.5, 3.7)).toBe(6.2);
  

In [41]:
failed_res = {k: {sub_k: sub_v for sub_k, sub_v in v.items() if not sub_v} for k, v in res.items()}
# remove {} from values
failed_res = {k: v for k, v in failed_res.items() if v}

print(failed_res)

{'multiply': {'should multiply two positive numbers correctly': False, 'should multiply two negative numbers correctly': False, 'should multiply a positive and a negative number correctly': False, 'should multiply two zero numbers correctly': False, 'should multiply a number with zero correctly': False, 'should multiply a number with one correctly': False, 'should multiply large numbers correctly': False, 'should multiply decimal numbers correctly': False, 'should return NaN when multiplying NaN values': False, 'should return Infinity when multiplying Infinity values': False, 'should return -Infinity when multiplying -Infinity values': False}}


In [68]:
failed_context = ""
failed_details = [] # (line_start, line_finish)

for suite_name in failed_res:
    print("Suite:" + suite_name)
    describe_pattern = r"describe\(['\"]"

    i = 0
    while i < len(jest_lines):
        match = re.search(describe_pattern + re.escape(suite_name), jest_lines[i])
        i+=1
        if match:
            break

    if i == len(jest_lines):
        print("Suite not found")
        continue

    start = match.start()
    end = match.end()
    
    for desc, _ in failed_res[suite_name].items():
        start_index = None
        end_index = None

        for j in range(i, len(jest_lines)):
            it_match = re.search(r"it\(['\"]" + re.escape(desc), jest_lines[j])
            if it_match:
                start_index = j

        if start_index == None: continue

        for j in range(start_index, len(jest_lines)):
            failed_context += jest_lines[j] + '\n'
            # print(jest_lines[j])
            if jest_lines[j].startswith('  });'):
                end_index = j
                break
                
        failed_details.append((start_index, end_index))

print(failed_context)

Suite:multiply
  it('should multiply two positive numbers correctly', () => {
    expect(multiply(2, 3)).toBe(6);
    expect(multiply(4, 5)).toBe(20);
  });
  it('should multiply two negative numbers correctly', () => {
    expect(multiply(-2, -3)).toBe(6);
    expect(multiply(-4, -5)).toBe(20);
  });
  it('should multiply a positive and a negative number correctly', () => {
    expect(multiply(2, -3)).toBe(-6);
    expect(multiply(-4, 5)).toBe(-20);
  });
  it('should multiply large numbers correctly', () => {
    expect(multiply(1000, 2000)).toBe(2000000);
    expect(multiply(-1000, 2000)).toBe(-2000000);
  });
  it('should multiply decimal numbers correctly', () => {
    expect(multiply(2.5, 3.5)).toBeCloseTo(8.75);
    expect(multiply(-2.5, 3.5)).toBeCloseTo(-8.75);
  });



In [69]:
failed_details

[(54, 57), (60, 63), (66, 69), (89, 92), (83, 86)]